In [1]:
import numpy as np
import networkx as nx
import requests
import time
from io import StringIO
from dimod import BinaryQuadraticModel
from dwave.system import LeapHybridSampler
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
class MaxCutQUBOSolver:
    """
    A class to solve Max Cut problems using QUBO formulation on D-Wave hybrid solvers
    """
    
    def __init__(self, api_token=None):
        """
        Initialize the solver with D-Wave API token
        
        Args:
            api_token (str): D-Wave API token. If None, will try to use environment variable
        """
        self.api_token = api_token
        self.sampler = LeapHybridSampler(token=api_token) if api_token else LeapHybridSampler()
        self.graph = None
        self.qubo = None
        self.solution = None
        self.results = {}
        
    def load_gset_graph(self, graph_name):
        """
        Load a graph from the G-set dataset
        
        Args:
            graph_name (str): Name of the graph (e.g., 'G1', 'G14', 'G22', etc.)
            
        Returns:
            networkx.Graph: The loaded graph
        """
        # Store the graph name for quality evaluation
        self.current_graph_name = graph_name
        
        # G-set dataset URLs (Stanford network analysis project mirror)
        base_url = "https://web.stanford.edu/~yyye/yyye/Gset/"
        
        graph_urls = {
            'G1': f"{base_url}G1",
            'G14': f"{base_url}G14", 
            'G22': f"{base_url}G22",
            'G49': f"{base_url}G49",
            'G50': f"{base_url}G50",
            'G55': f"{base_url}G55",
            'G70': f"{base_url}G70"
        }
        
        if graph_name not in graph_urls:
            raise ValueError(f"Graph {graph_name} not available. Available graphs: {list(graph_urls.keys())}")
        
        try:
            print(f"Loading graph {graph_name} from G-set dataset...")
            response = requests.get(graph_urls[graph_name])
            response.raise_for_status()
            
            # Parse the graph file
            lines = response.text.strip().split('\n')
            
            # First line contains number of vertices and edges
            first_line = lines[0].split()
            n_vertices = int(first_line[0])
            n_edges = int(first_line[1])
            
            print(f"Graph {graph_name}: {n_vertices} vertices, {n_edges} edges")
            
            # Create NetworkX graph
            G = nx.Graph()
            G.add_nodes_from(range(1, n_vertices + 1))  # G-set uses 1-based indexing
            
            # Add edges with weights
            for line in lines[1:]:
                if line.strip():  # Skip empty lines
                    parts = line.split()
                    if len(parts) >= 3:
                        u, v, weight = int(parts[0]), int(parts[1]), float(parts[2])
                        G.add_edge(u, v, weight=weight)
            
            self.graph = G
            print(f"Successfully loaded {graph_name}")
            return G
            
        except requests.RequestException as e:
            print(f"Error downloading {graph_name}: {e}")
            # Fallback: create a sample graph for testing
            print("Creating sample graph for testing...")
            return self._create_sample_graph(graph_name)
        
    def _create_sample_graph(self, name):
        """Create a sample graph for testing when download fails"""
        if name == 'G1':
            G = nx.Graph()
            G.add_weighted_edges_from([
                (1, 2, 1), (1, 3, 1), (2, 3, 1), (2, 4, 1), 
                (3, 4, 1), (1, 4, 1), (1, 5, 1), (5, 2, 1)
            ])
        else:
            # Create a random graph for other cases
            np.random.seed(42)
            n = 20 if name == 'G14' else 10
            G = nx.erdos_renyi_graph(n, 0.3, seed=42)
            # Convert to 1-based indexing and add random weights
            G = nx.relabel_nodes(G, {i: i+1 for i in range(n)})
            for u, v in G.edges():
                G[u][v]['weight'] = np.random.randint(1, 10)
        
        self.graph = G
        self.current_graph_name = name
        print(f"Created sample graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
        return G
    
    def formulate_qubo(self):
        """
        Formulate the Max Cut problem as a QUBO
        
        The QUBO formulation for Max Cut:
        - Decision variables: x_i ∈ {0,1} for each vertex i
        - x_i = 1 means vertex i is in set S, x_i = 0 means vertex i is in set T
        - Objective: Maximize Σ w_ij * (x_i + x_j - 2*x_i*x_j) for all edges (i,j)
        - This is equivalent to minimizing: Σ w_ij * (2*x_i*x_j - x_i - x_j)
        """
        if self.graph is None:
            raise ValueError("No graph loaded. Please load a graph first.")
        
        print("Formulating QUBO...")
        
        # Initialize QUBO dictionary
        Q = defaultdict(float)
        
        # Convert to minimization problem (D-Wave minimizes by default)
        # We want to maximize the cut, so we minimize the negative cut
        total_weight = sum(data.get('weight', 1) for _, _, data in self.graph.edges(data=True))
        
        for u, v, data in self.graph.edges(data=True):
            weight = data.get('weight', 1)
            
            # Add linear terms: -weight * (x_u + x_v)
            Q[(u, u)] -= weight
            Q[(v, v)] -= weight
            
            # Add quadratic term: 2 * weight * x_u * x_v
            Q[(u, v)] += 2 * weight
        
        # Convert to BinaryQuadraticModel
        self.qubo = BinaryQuadraticModel.from_qubo(Q)
        
        print(f"QUBO formulated with {len(self.qubo.variables)} variables")
        print(f"Linear terms: {len(self.qubo.linear)}")
        print(f"Quadratic terms: {len(self.qubo.quadratic)}")
        
        return self.qubo
    
    def solve(self, time_limit=10):
        """
        Solve the QUBO using D-Wave hybrid solver
        
        Args:
            time_limit (int): Time limit in seconds for the hybrid solver
            
        Returns:
            dict: Solution results including cut value, partition, and timing
        """
        if self.qubo is None:
            raise ValueError("QUBO not formulated. Please call formulate_qubo() first.")
        
        print(f"Solving QUBO using D-Wave Hybrid Solver (time limit: {time_limit}s)...")
        
        start_time = time.time()
        
        try:
            # Submit to D-Wave hybrid solver
            sampleset = self.sampler.sample(
                self.qubo,
                time_limit=time_limit,
                label=f"Max Cut Problem - {self.graph.number_of_nodes()} nodes"
            )
            
            solve_time = time.time() - start_time
            
            # Get the best solution
            best_sample = sampleset.first.sample
            best_energy = sampleset.first.energy
            
            # Calculate the actual cut value (remember we minimized the negative)
            cut_value = self._calculate_cut_value(best_sample)
            
            # Create partition
            set_s = [node for node, value in best_sample.items() if value == 1]
            set_t = [node for node, value in best_sample.items() if value == 0]
            
            # Store results
            self.results = {
                'cut_value': cut_value,
                'partition': (set_s, set_t),
                'energy': best_energy,
                'solve_time': solve_time,
                'num_variables': len(self.qubo.variables),
                'num_edges': self.graph.number_of_edges(),
                'sampleset': sampleset
            }
            
            self.solution = best_sample
            
            print(f"Solution found!")
            print(f"Cut value: {cut_value}")
            print(f"Solving time: {solve_time:.3f} seconds")
            print(f"Set S: {len(set_s)} nodes")
            print(f"Set T: {len(set_t)} nodes")
            
            return self.results
            
        except Exception as e:
            print(f"Error solving QUBO: {e}")
            return None
    
    def _calculate_cut_value(self, solution):
        """Calculate the cut value for a given solution"""
        cut_value = 0
        for u, v, data in self.graph.edges(data=True):
            weight = data.get('weight', 1)
            # Edge contributes to cut if endpoints are in different sets
            if solution[u] != solution[v]:
                cut_value += weight
        return cut_value
    
    def evaluate_solution_quality(self):
        """
        Evaluate the quality of the solution by comparing with best known cut values
        """
        if self.solution is None:
            print("No solution available. Please solve first.")
            return
        
        print("\n=== Solution Quality Evaluation ===")
        
        # Best known cut values for G-set instances (from literature)
        best_known_cuts = {
            'G1': 11624,    # 800 nodes
            'G2': 11620,    # 800 nodes  
            'G3': 11622,    # 800 nodes
            'G4': 11646,    # 800 nodes
            'G5': 11631,    # 800 nodes
            'G6': 2178,     # 800 nodes, unweighted
            'G7': 2006,     # 800 nodes, unweighted
            'G8': 2005,     # 800 nodes, unweighted
            'G9': 2054,     # 800 nodes, unweighted
            'G10': 2000,    # 800 nodes, unweighted
            'G11': 564,     # 800 nodes, unweighted
            'G12': 556,     # 800 nodes, unweighted
            'G13': 582,     # 800 nodes, unweighted
            'G14': 3064,    # 800 nodes, unweighted
            'G15': 3050,    # 800 nodes, unweighted
            'G16': 3052,    # 800 nodes, unweighted
            'G17': 3047,    # 800 nodes, unweighted
            'G18': 992,     # 800 nodes, unweighted
            'G19': 906,     # 800 nodes, unweighted
            'G20': 941,     # 800 nodes, unweighted
            'G21': 931,     # 800 nodes, unweighted
            'G22': 13359,   # 2000 nodes
            'G23': 13344,   # 2000 nodes
            'G24': 13337,   # 2000 nodes
            'G25': 13340,   # 2000 nodes
            'G26': 13328,   # 2000 nodes
            'G27': 3341,    # 2000 nodes, unweighted
            'G28': 3298,    # 2000 nodes, unweighted
            'G29': 3405,    # 2000 nodes, unweighted
            'G30': 3421,    # 2000 nodes, unweighted
            'G31': 3288,    # 2000 nodes, unweighted
            'G32': 1410,    # 2000 nodes, unweighted
            'G33': 1382,    # 2000 nodes, unweighted
            'G34': 1384,    # 2000 nodes, unweighted
            'G35': 7687,    # 2000 nodes, unweighted
            'G36': 7680,    # 2000 nodes, unweighted
            'G37': 7691,    # 2000 nodes, unweighted
            'G38': 7688,    # 2000 nodes, unweighted
            'G39': 2408,    # 2000 nodes, unweighted
            'G40': 2400,    # 2000 nodes, unweighted
            'G41': 2405,    # 2000 nodes, unweighted
            'G42': 2481,    # 2000 nodes, unweighted
            'G43': 6660,    # 1000 nodes
            'G44': 6650,    # 1000 nodes
            'G45': 6654,    # 1000 nodes
            'G46': 6649,    # 1000 nodes
            'G47': 6657,    # 1000 nodes
            'G48': 6000,    # 3000 nodes, unweighted
            'G49': 6000,    # 3000 nodes, unweighted
            'G50': 5880,    # 3000 nodes, unweighted
            'G51': 4012,    # 1000 nodes, unweighted
            'G52': 4008,    # 1000 nodes, unweighted
            'G53': 4013,    # 1000 nodes, unweighted
            'G54': 4030,    # 1000 nodes, unweighted
            'G55': 10299,   # 5000 nodes, unweighted
            'G56': 4016,    # 1000 nodes, unweighted
            'G57': 3494,    # 1000 nodes, unweighted
            'G58': 19276,   # 5000 nodes, unweighted
            'G59': 8735,    # 5000 nodes, unweighted
            'G60': 14186,   # 7000 nodes, unweighted
            'G61': 5804,    # 5000 nodes, unweighted
            'G62': 6885,    # 5000 nodes, unweighted
            'G63': 8016,    # 5000 nodes, unweighted
            'G64': 8051,    # 5000 nodes, unweighted
            'G65': 5560,    # 5000 nodes, unweighted
            'G66': 6364,    # 5000 nodes, unweighted
            'G67': 6944,    # 10000 nodes, unweighted
            'G70': 9541,    # 10000 nodes, unweighted
        }
        
        # Determine which graph we're working with
        graph_name = getattr(self, 'current_graph_name', 'Unknown')
        
        # Our solution
        our_cut = self.results['cut_value']
        print(f"Our solution cut value: {our_cut}")
        
        # Compare with best known solution
        if graph_name in best_known_cuts:
            best_known = best_known_cuts[graph_name]
            print(f"Best known cut value: {best_known}")
            
            # Quality metrics
            approximation_ratio = our_cut / best_known
            gap = best_known - our_cut
            gap_percentage = (gap / best_known) * 100
            
            print(f"Approximation ratio: {approximation_ratio:.4f}")
            print(f"Gap from optimum: {gap} ({gap_percentage:.2f}%)")
            
            if approximation_ratio >= 0.95:
                print("✓ Excellent solution quality (≥95% of best known)")
            elif approximation_ratio >= 0.90:
                print("✓ Good solution quality (≥90% of best known)")
            elif approximation_ratio >= 0.80:
                print("○ Fair solution quality (≥80% of best known)")
            else:
                print("× Poor solution quality (<80% of best known)")
                
        else:
            print(f"Best known cut value for {graph_name}: Not available in database")
            print("Cannot evaluate solution quality without reference value")
        
    def visualize_solution(self, figsize=(12, 8)):
        """Visualize the graph with the Max Cut solution"""
        if self.solution is None:
            print("No solution available. Please solve first.")
            return
        
        plt.figure(figsize=figsize)
        
        # Create position layout
        pos = nx.spring_layout(self.graph, seed=42)
        
        # Separate nodes by partition
        set_s, set_t = self.results['partition']
        
        # Draw nodes
        nx.draw_networkx_nodes(self.graph, pos, nodelist=set_s, 
                              node_color='lightblue', label='Set S', 
                              node_size=300, alpha=0.8)
        nx.draw_networkx_nodes(self.graph, pos, nodelist=set_t, 
                              node_color='lightcoral', label='Set T', 
                              node_size=300, alpha=0.8)
        
        # Draw edges
        cut_edges = []
        non_cut_edges = []
        
        for u, v in self.graph.edges():
            if self.solution[u] != self.solution[v]:
                cut_edges.append((u, v))
            else:
                non_cut_edges.append((u, v))
        
        # Draw cut edges (thick red)
        nx.draw_networkx_edges(self.graph, pos, edgelist=cut_edges, 
                              edge_color='red', width=2, alpha=0.8)
        
        # Draw non-cut edges (thin gray)
        nx.draw_networkx_edges(self.graph, pos, edgelist=non_cut_edges, 
                              edge_color='gray', width=1, alpha=0.5)
        
        # Add labels
        nx.draw_networkx_labels(self.graph, pos, font_size=8)
        
        plt.title(f'Max Cut Solution\nCut Value: {self.results["cut_value"]}, '
                 f'Cut Edges: {len(cut_edges)}/{self.graph.number_of_edges()}')
        plt.legend()
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    def print_detailed_results(self):
        """Print detailed results of the solution"""
        if self.results is None:
            print("No results available. Please solve first.")
            return
        
        print("\n" + "="*50)
        print("DETAILED RESULTS")
        print("="*50)
        
        print(f"Graph Statistics:")
        print(f"  - Nodes: {self.graph.number_of_nodes()}")
        print(f"  - Edges: {self.graph.number_of_edges()}")
        
        print(f"\nQUBO Statistics:")
        print(f"  - Variables: {self.results['num_variables']}")
        print(f"  - Linear terms: {len(self.qubo.linear)}")
        print(f"  - Quadratic terms: {len(self.qubo.quadratic)}")
        
        print(f"\nSolution Quality:")
        print(f"  - Cut value: {self.results['cut_value']}")
        print(f"  - QUBO energy: {self.results['energy']}")
        print(f"  - Partition sizes: {len(self.results['partition'][0])} | {len(self.results['partition'][1])}")
        
        print(f"\nTiming:")
        print(f"  - Solve time: {self.results['solve_time']:.3f} seconds")
        
        print(f"\nD-Wave Solver Info:")
        sampleset = self.results['sampleset']
        print(f"  - Total samples: {len(sampleset)}")
        print(f"  - Best energy: {sampleset.first.energy}")
        if hasattr(sampleset, 'info'):
            print(f"  - Solver info: {sampleset.info}")

# Example usage and testing
def run_maxcut_experiment(graph_names=['G1'], api_token=None):
    """
    Run Max Cut experiments on specified graphs
    
    Args:
        graph_names (list): List of graph names to test
        api_token (str): D-Wave API token
    """
    results_summary = []
    
    for graph_name in graph_names:
        print(f"\n{'='*60}")
        print(f"EXPERIMENT: {graph_name}")
        print(f"{'='*60}")
        
        # Initialize solver
        solver = MaxCutQUBOSolver(api_token=api_token)
        
        try:
            # Load graph
            graph = solver.load_gset_graph(graph_name)
            
            # Formulate QUBO
            qubo = solver.formulate_qubo()
            
            # Solve
            results = solver.solve(time_limit=20)
            
            if results:
                # Evaluate and visualize
                solver.evaluate_solution_quality()
                solver.print_detailed_results()
                
                # Store summary
                results_summary.append({
                    'graph': graph_name,
                    'nodes': graph.number_of_nodes(),
                    'edges': graph.number_of_edges(),
                    'cut_value': results['cut_value'],
                    'solve_time': results['solve_time']
                })
                
                # Visualize (comment out if running in non-interactive environment)
                # solver.visualize_solution()
            
        except Exception as e:
            print(f"Error processing {graph_name}: {e}")
    
    # Print summary
    if results_summary:
        print(f"\n{'='*60}")
        print("EXPERIMENT SUMMARY")
        print(f"{'='*60}")
        print(f"{'Graph':<10} {'Nodes':<8} {'Edges':<8} {'Cut Value':<12} {'Time (s)':<10}")
        print("-" * 60)
        for result in results_summary:
            print(f"{result['graph']:<10} {result['nodes']:<8} {result['edges']:<8} "
                  f"{result['cut_value']:<12} {result['solve_time']:<10.3f}")

In [3]:
# Example usage:
if __name__ == "__main__":
    # Set your D-Wave API token here
    API_TOKEN = "UcZ6-53536c1d213e187243eaa16383307908db300998"  # Replace with your actual token
    
    # Run experiments on selected graphs
    print("Starting Max Cut QUBO experiments with D-Wave Hybrid Solver...")
    
    # Test with smaller graphs first
    run_maxcut_experiment(['G22'], api_token=API_TOKEN)
    
    print("\nExperiment completed!")

Starting Max Cut QUBO experiments with D-Wave Hybrid Solver...

EXPERIMENT: G22
Loading graph G22 from G-set dataset...
Graph G22: 2000 vertices, 19990 edges
Successfully loaded G22
Formulating QUBO...
QUBO formulated with 2000 variables
Linear terms: 2000
Quadratic terms: 19990
Solving QUBO using D-Wave Hybrid Solver (time limit: 20s)...
Solution found!
Cut value: 13359.0
Solving time: 6.452 seconds
Set S: 1005 nodes
Set T: 995 nodes

=== Solution Quality Evaluation ===
Our solution cut value: 13359.0
Best known cut value: 13359
Approximation ratio: 1.0000
Gap from optimum: 0.0 (0.00%)
✓ Excellent solution quality (≥95% of best known)

DETAILED RESULTS
Graph Statistics:
  - Nodes: 2000
  - Edges: 19990

QUBO Statistics:
  - Variables: 2000
  - Linear terms: 2000
  - Quadratic terms: 19990

Solution Quality:
  - Cut value: 13359.0
  - QUBO energy: -13359.0
  - Partition sizes: 1005 | 995

Timing:
  - Solve time: 6.452 seconds

D-Wave Solver Info:
  - Total samples: 1
  - Best energy: -